In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
File: s2_test.py
Version: v1.0
Date: 2023-02-09
Authors: Chen G.
Description: This script creates downloading and processing Sentinel-2 images based on Google Earth Engine.
License: This code is distributed under the MIT License.

    Parameter:
        START_DATE: The earliest date to include images for (inclusive).
        END_DATE: The latest date to include images for (exclusive).
        BANDS: The Sentinel-2 image bands to select for processing.
        ROI: The boundry to select for processing.
        MAX_CLOUD_PROBABILITY: (Optional) cloud cover percentage to apply s2 image collection filter.
        CAL_NDVI: (Optional) calculate the Normalized Difference Vegetation Index (NDVI) from multiband s2 images.
        CAL_NDMI: (Optional) calculate the Normalized Difference Moisture Index (NDMI) from multiband s2 images.
        CLIP_TO_ROI: (Optional) clip the processed image to the region of interest.
        SAVE_ASSETS : (Optional) exports the processed collection to an asset.
        ASSET_ID : (Optional) the user id path to save the assets
        SAVE_LOCAL : (Optional) download the processed images to local.
        VISUALIZATION : (Optional) convert raw image to RGB image and download the processed images to local.
        LOCAL_DIR : (Optional) where to save downloaded images.
        
    Returns:
        An ee.ImageCollection with an analysis ready Sentinel 2 imagery with the cloud masked images and vegetation index band.


    """

import base_process as bp


# Query Dates
dates_list_2019 = ['2019-01-24', '2019-03-12', '2019-03-15', '2019-03-17', '2019-04-01', '2019-04-06', '2019-04-09', '2019-05-04', 
                   '2019-05-09', '2019-05-11', '2019-05-24', '2019-06-15', '2019-06-28', '2019-07-23', '2019-07-25', '2019-07-28', 
                   '2019-07-30', '2019-08-12', '2019-08-14', '2019-08-17', '2019-08-22', '2019-09-11', '2019-09-16', '2019-09-18', 
                   '2019-09-26', '2019-10-03', '2019-10-31', '2019-11-05', '2019-11-15', '2019-11-17', '2019-11-20', '2019-12-02', 
                   '2019-12-07', '2019-12-10', '2019-12-12', '2019-12-15', '2019-12-30']

dates_list_2020 = ['2020-01-06', '2020-01-21', '2020-01-29', '2020-01-31', '2020-02-03', '2020-02-05', '2020-02-10', '2020-02-18', 
                   '2020-02-20', '2020-02-23', '2020-03-06', '2020-03-14', '2020-03-16', '2020-03-19', '2020-03-21', '2020-03-31', 
                   '2020-04-05', '2020-04-13', '2020-04-15', '2020-04-23', '2020-04-25', '2020-04-28', '2020-05-03', '2020-05-13',
                   '2020-06-07', '2020-08-01', '2020-08-13', '2020-08-16', '2020-08-23', '2020-08-28', '2020-09-05', '2020-09-07', 
                   '2020-09-25', '2020-10-02', '2020-10-17', '2020-10-20', '2020-10-30', '2020-11-04', '2020-11-06', '2020-11-11', 
                   '2020-11-29', '2020-12-01', '2020-12-04', '2020-12-09', '2020-12-16', '2020-12-21', '2020-12-31']

dates_list_2021 = ['2021-01-08', '2021-01-10', '2021-01-13', '2021-01-15', '2021-01-18', '2021-01-25', '2021-01-30', '2021-02-04', 
                   '2021-02-19', '2021-02-22', '2021-03-14', '2021-03-29', '2021-04-10', '2021-04-25', '2021-04-30', '2021-05-05', 
                   '2021-05-10', '2021-05-25', '2021-06-07', '2021-06-24', '2021-07-07', '2021-07-14', '2021-07-22', '2021-08-28', 
                   '2021-09-20', '2021-09-22', '2021-10-05', '2021-10-10', '2021-10-25', '2021-11-09', '2021-11-11', '2021-11-14', 
                   '2021-11-16', '2021-11-19', '2021-11-24', '2021-11-26', '2021-12-04', '2021-12-06', '2021-12-11', '2021-12-19', 
                   '2021-12-26', '2021-12-29', '2021-12-31']

dates_list_2022 = ['2022-01-03', '2022-01-18', '2022-02-04', '2022-02-14', '2022-02-27', '2022-03-09', '2022-03-14', '2022-03-24',  
                   '2022-03-26', '2022-03-29', '2022-04-03', '2022-04-08', '2022-04-10', '2022-04-20', '2022-05-03', '2022-05-05', 
                   '2022-05-08', '2022-05-18', '2022-06-22', '2022-07-04', '2022-07-07', '2022-07-12', '2022-07-14', '2022-07-29', 
                   '2022-08-03', '2022-08-06', '2022-08-08', '2022-08-11', '2022-08-16', '2022-08-23', '2022-09-17', '2022-09-20', 
                   '2022-09-27', '2022-10-02', '2022-10-10', '2022-10-25', '2022-10-30', '2022-11-01', '2022-11-06', '2022-11-16', 
                   '2022-11-21', '2022-12-06', '2022-12-14', '2022-12-21', '2022-12-24', '2022-12-31']

dates_list_2023 = ['2023-01-03', '2023-01-08', '2023-01-18', '2023-01-20', '2023-01-25', '2023-01-28', '2023-01-30', '2023-03-04',  
                   '2023-03-06', '2023-03-09', '2023-03-14', '2023-03-16', '2023-03-26', '2023-04-03', '2023-04-28', '2023-04-30',
                   '2023-05-13', '2023-05-15', '2023-05-25', '2023-05-28', '2023-05-30']


for d in dates_list_2023:
    # processed s2 collection
    bp.lingang_preprocess(d, "EPSG:4326", "G:/geoserver_data/ndvi_wgs84", "G:/geoserver_data/ndvi_gcj02")


lingang_DT_ZS_20230103.tif: |          | 0.00/643k (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


lingang_DT_ZS_20230108.tif: |          | 0.00/643k (raw) [  0.0%] in 00:00 (eta:     ?)

lingang_DT_ZS_20230118.tif: |          | 0.00/643k (raw) [  0.0%] in 00:00 (eta:     ?)

lingang_DT_ZS_20230120.tif: |          | 0.00/643k (raw) [  0.0%] in 00:00 (eta:     ?)

...

lingang_DT_ZS_20230513.tif: |          | 0.00/643k (raw) [  0.0%] in 00:00 (eta:     ?)

lingang_DT_ZS_20230515.tif: |          | 0.00/643k (raw) [  0.0%] in 00:00 (eta:     ?)

lingang_DT_ZS_20230525.tif: |          | 0.00/643k (raw) [  0.0%] in 00:00 (eta:     ?)

lingang_DT_ZS_20230528.tif: |          | 0.00/643k (raw) [  0.0%] in 00:00 (eta:     ?)

lingang_DT_ZS_20230530.tif: |          | 0.00/643k (raw) [  0.0%] in 00:00 (eta:     ?)